In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from models import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
activation = hard_sigmoid
# activation = F.relu
criterion = torch.nn.MSELoss(reduction='none').to(device)

In [5]:
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_h = 0.9999
lambda_y = 0.9999
epsilon = 0.1
one_over_epsilon = 1 / epsilon
lr_start = {'ff' : 0.15, 'fb': 0.15, 'lat': 1e-3}
neural_lr = 0.02
model = TwoLayerCorInfoMaxThreePhase(architecture = architecture, lambda_h = lambda_h, lambda_y = lambda_y, 
                                     epsilon = epsilon, activation = activation)

In [ ]:
trn_acc_list = []
tst_acc_list = []
neural_dynamic_iterations_free = 30
neural_dynamic_iterations_nudged = 10
# lambda_h = 0.01
# lambda_y = 0.01
# epsilon = 1
# one_over_epsilon = 1 / epsilon
n_epochs = 50
# lr = {'ff' : 1e-3, 'fb': 1e-3, 'lat': 1e-3}
# neural_lr = 0.25

for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.7)**epoch_, 'fb' : lr_start['fb'] * (0.7)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T

        h, y_hat = model.batch_step(  x, y_one_hot, lr, neural_lr, neural_dynamic_iterations_free, 
                                      neural_dynamic_iterations_nudged, beta)
    
    trn_acc = evaluateCorInfoMax(model, train_loader, neural_lr, 20, device = 'cuda', printing = False)
    tst_acc = evaluateCorInfoMax(model, test_loader, neural_lr, 20, device = 'cuda', printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
#     lr = {'ff' : 0.05 * (0.9)**epoch_, 'fb': 0.05 * (0.9) ** epoch_, 'lat': 1e-3}
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [01:04, 46.45it/s]
5it [00:00, 47.01it/s]

Epoch : 1, Train Accuracy : 0.8995833333333333, Test Accuracy : 0.9067


3000it [01:03, 47.58it/s]
5it [00:00, 47.67it/s]

Epoch : 2, Train Accuracy : 0.9169166666666667, Test Accuracy : 0.9216


3000it [01:03, 47.30it/s]
5it [00:00, 49.71it/s]

...

3000it [01:02, 47.87it/s]
5it [00:00, 49.80it/s]

Epoch : 13, Train Accuracy : 0.9361333333333334, Test Accuracy : 0.9365


3000it [01:03, 47.16it/s]
5it [00:00, 49.75it/s]

Epoch : 14, Train Accuracy : 0.9362, Test Accuracy : 0.9365


1446it [00:29, 48.45it/s]